# Importing

In [58]:
import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Getting the training set

In [59]:
intents = json.loads(open('D:/100 days of Python/Chat Bot/intents.json').read())

In [60]:
intents

{'intents': [{'tag': 'greetings',
   'patterns': ['hello',
    'hi',
    'goo day',
    'hey',
    "what's up",
    "how's it going"],
   'responses': ['Hello!',
    'How may I assist you today?',
    'What can I do for you']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'goodbye',
    'see u',
    'I have to go now',
    'gotta bounce',
    'talk to you later'],
   'responses': ['Goodbye',
    'Surely! Have a great day',
    "We'll talk later",
    'It was great talking to you! Bye',
    'Have a good day']},
  {'tag': 'creater',
   'patterns': ['Who created you?',
    'Who developed you?',
    'Who is your owner?'],
   'responses': ['I was developed and trained by Faiq Ahmad.',
    'My owner is Faiq Ahmad']},
  {'tag': 'name',
   'patterns': ['What is your name?',
    "Who're u?",
    'Tell me about yourself',
    'What should I call you?',
    'Can you tell me your name?'],
   'responses': ["My name is Aleeza the Chatbot! and I'm a chat bot!"]},
  {'tag': 'age',
   'patterns': ['

In [61]:
lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ",", ","]

In [62]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)
print(classes)
# print(words)

[(['hello'], 'greetings'), (['hi'], 'greetings'), (['goo', 'day'], 'greetings'), (['hey'], 'greetings'), (['what', "'s", 'up'], 'greetings'), (['how', "'s", 'it', 'going'], 'greetings'), (['Bye'], 'goodbye'), (['goodbye'], 'goodbye'), (['see', 'u'], 'goodbye'), (['I', 'have', 'to', 'go', 'now'], 'goodbye'), (['got', 'ta', 'bounce'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['Who', 'created', 'you', '?'], 'creater'), (['Who', 'developed', 'you', '?'], 'creater'), (['Who', 'is', 'your', 'owner', '?'], 'creater'), (['What', 'is', 'your', 'name', '?'], 'name'), (['Who', "'re", 'u', '?'], 'name'), (['Tell', 'me', 'about', 'yourself'], 'name'), (['What', 'should', 'I', 'call', 'you', '?'], 'name'), (['Can', 'you', 'tell', 'me', 'your', 'name', '?'], 'name'), (['How', 'old', 'are', 'you', '?'], 'age'), (['What', "'s", 'your', 'age', '?'], 'age'), (['Can', 'you', 'tell', 'me', 'your', 'age', '?'], 'age'), (['age', '?'], 'age'), (['I', "'m", 'feeling', 'sad', 'today'], 'mood'),

### Converting words into their base words and then removing duplicate and sorting them

In [63]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

In [64]:
words = sorted(set(words))

In [65]:
classes = sorted(set(classes))

In [66]:
pickle.dump(words, open('words.pkl', 'wb'))

In [67]:
pickle.dump(classes, open('classes.pkl', 'wb'))

In [68]:
training = []
output_empty = [0] * len(classes)

In [69]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [70]:
random.shuffle(training)
training = np.array(training)

C:\Users\Dell\AppData\Local\Temp\ipykernel_12656\2195589081.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [71]:
train_x = list(training[:,0])
train_y = list(training[:,1])

### Neural Network Model

In [72]:
from tensorflow.keras.optimizers import legacy
model = Sequential()

In [73]:
model.add(Dense(128, activation = 'relu', input_shape = (len(train_x[0]),)))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))
sgd = legacy.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = 'accuracy')

In [75]:
model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)
model.save('chatbotmodel.h5')
print("Done")

Epoch 1/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0487 - accuracy: 0.9762
Epoch 2/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0359 - accuracy: 0.9762
Epoch 3/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0392 - accuracy: 0.9762
Epoch 4/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0214 - accuracy: 1.0000
Epoch 5/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0236 - accuracy: 1.0000
Epoch 6/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0099 - accuracy: 1.0000
Epoch 7/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0413 - accuracy: 1.0000
Epoch 8/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0200 - accuracy: 1.0000
Epoch 9/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0381 - accuracy: 1.0000
Epoch 10/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0703 - accuracy: 1.0000
Epoch 11/200
9/9 [===========

9/9 [==============================] - 0s 3ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 85/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0208 - accuracy: 1.0000
Epoch 86/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0187 - accuracy: 1.0000
Epoch 87/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0153 - accuracy: 1.0000
Epoch 88/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 89/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0286 - accuracy: 1.0000
Epoch 90/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 91/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0502 - accuracy: 1.0000
Epoch 92/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 93/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 94/200
9/9 [===============

9/9 [==============================] - 0s 2ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 168/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0129 - accuracy: 1.0000
Epoch 169/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0856 - accuracy: 0.9524
Epoch 170/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 171/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0082 - accuracy: 1.0000
Epoch 172/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 173/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0736 - accuracy: 0.9762
Epoch 174/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0226 - accuracy: 1.0000
Epoch 175/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0116 - accuracy: 1.0000
Epoch 176/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 177/200
9/9 [=====